In [359]:
from contextlib import contextmanager
from threading  import Lock, Thread
import random
from time import sleep
import operator as _

class SharedExclusive(object):

    def __init__(self):
        self.write_lock_obj = Lock()
        self.num_r = 0 # Number of read_locks

    def r_acquire(self):
        print("\nRead_lock Called:")
        if self.write_lock_obj.locked():
            print("Wait for write_lock")
            
            # whaiting until write_lock released...
            while self.write_lock_obj.locked():
                print(.,end="")
            print("Read lock")
            self.num_r += 1
        else:
            print("Read lock")
            self.num_r += 1

    def r_release(self):
        if self.num_r > 0:
            print("Release read lock")
            self.num_r -= 1

    @contextmanager
    def r_locked(self):
        try:
            self.r_acquire()
            yield
        finally:
            self.r_release()

    def w_acquire(self):
        print("\nWrite_lock Called:")
        if self.num_r:
            print("Wait for read_lock")
            
            # whaiting until read_lock released...
            while self.num_r:
                print(*,end="")
                pass
            print('Write lock')
            self.write_lock_obj.acquire()
            
        elif self.write_lock_obj.locked():
            print("Wait for write_lock")
            self.write_lock_obj.acquire()
        else:
            print('Write lock')
            self.write_lock_obj.acquire()

    def w_release(self):
        if self.write_lock_obj.locked():
            print('Release write lock')
            self.write_lock_obj.release()

    @contextmanager
    def w_locked(self):
        try:
            self.w_acquire()
            yield
        finally:
            self.w_release()
    
    def create_local_variable(self, block):
        variable = 0
        ldic = locals()
        try:
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} already exists!")
        except AttributeError:
            exec(f"self.{block['variable']} = 0")
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} created!")
        finally:
            variable = ldic['variable']
        return variable
    
    def create_transaction(self,transaction):
        for block in transaction:
            operation_name = block['operations'][0]
            if operation_name == 'read_item':
                with self.r_locked():
                    variable = self.create_local_variable(block)
                    print(variable)
                    random_number = random.uniform(0.1,1.5)
                    print(f"~~ Sleep for {random_number:.2f} s")
                    sleep(random_number)
                    
            elif operation_name == 'write_item':
                operation_task = block['operations'][1]
                with self.w_locked():
                    variable = self.create_local_variable(block)
                    print(variable)
                    operator, operand = operation_task.split()
                    mapper = {
                        '+' : _.add,
                        '-' : _.sub,
                        '*' : _.mul,
                        '/' : _.truediv,
                    }
                    func = mapper.get(operator, None)
                    if func:
                        variable = func(variable, float(operand))
                    random_number = random.uniform(0.1,1.5)
                    print(f"~~ Sleep for {random_number:.2f} s")
                    sleep(random_number)
                    exec(f"self.{block['variable']} = variable")
                

SyntaxError: invalid syntax (356283532.py, line 20)

In [357]:
def create_and_run_threads(schedule, lock):
    for index, transaction in enumerate(schedule):
        exec(f"t{index} = Thread(target=lock.create_transaction, args=({transaction},))")
    for i in range(index + 1):
        exec(f"t{i}.start()")
    for i in range(index + 1):
        exec(f"t{i}.join()")

def print_variables(schedule, lock):
    print('\nAll used variables and values:')
    vars = []
    for transaction in schedule:
        for block in transaction:
            vars.append(block['variable'])
    # Remove duplicates
    vars = list(set(vars))
    for var in vars:
        print(var, "=", getattr(lock, var))
        
def main():
    lock = SharedExclusive()
    transaction = [
        {
            'variable': "x",
            'operations': [
                'read_item', 
            ]
        },
        {
            'variable': "y",
            'operations': [
                'write_item', 
                '+ 10'
            ]
        },
        {
            'variable': "y",
            'operations': [
                'read_item', 
            ]
        },
        {
            'variable': "x",
            'operations': [
                'write_item', 
                '+ 10'
            ]
        },
    ]
    # for simplicity we assume all transactions are same.
    schedule = [transaction for _ in range(2)]
    create_and_run_threads(schedule, lock)
    print_variables(schedule, lock)

In [358]:
main()


Read_lock Called:
Read lock
0
~~ Sleep for 0.83 s

Read_lock Called:
Read lock
0
~~ Sleep for 1.15 s
Release read lock

Write_lock Called:
Wait for read_lock
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222